#Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Imports

In [2]:
import os
import json
import imageio
import csv
import numpy as np
from operator import itemgetter

from keras.utils import to_categorical
from keras.models import load_model
from tensorflow.keras.metrics import MeanIoU

import matplotlib.pyplot as plt
import matplotlib

from scipy.stats import mode
import time

import sys
sys.path.append("/content/drive/My Drive/tf-keras-SegNet")
from layers import MaxPoolingWithArgmax2D, MaxUnpooling2D
sys.path.append('/content/drive/My Drive')
from lasero.utils import training 

Using TensorFlow backend.


In [0]:
base_dir = "/content/drive/My Drive"

#Configurations and Parameters


In [43]:
dir_indices = 0, 0, -1
model_dir = os.path.join(base_dir, 'models')
input_components = [i for i in os.listdir(model_dir) if 'dataset' in i]
print(input_components)
selected_input_component = input_components[dir_indices[0]]
print(selected_input_component)
input_component_dir = os.path.join(model_dir, selected_input_component)
print()

opt_components = [i for i in os.listdir(input_component_dir)]
print(opt_components)
selected_opt_component = opt_components[dir_indices[1]]
print(selected_opt_component)
opt_component_dir = os.path.join(input_component_dir, selected_opt_component)
print()

callback_components = [i for i in os.listdir(opt_component_dir)]
print(callback_components)
selected_callback_component = callback_components[dir_indices[2]]
print(selected_callback_component)
print()

session_name = os.path.join(selected_input_component, selected_opt_component, selected_callback_component)

session_paths = training.create_session_paths(session_name)
for i in session_paths:
    print(session_paths[i])

print('\navailable models:')

for i in os.listdir(session_paths['session_dir']):
    print(i)

['dataset=one_percent_bands=1-4-8-_weights=none', 'dataset=one_percent_bands=1-2-3-_weights=none', 'dataset=one_percent_bands=0-1-2-3-4-5-6-7-8-9-_weights=none']
dataset=one_percent_bands=1-4-8-_weights=none

['opt=Adam_loss=categorical_crossentropy_lr=0.001']
opt=Adam_loss=categorical_crossentropy_lr=0.001

['batch=4_epochs=50_lrmonitor=val_loss_factor=0.5_patience=3_min=0.0001_stopmonitor=val_loss_patience=9', 'batch=5_epochs=50_lrmonitor=val_loss_factor=0.5_patience=3_min=0.0001_stopmonitor=val_loss_patience=9', 'batch=5_epochs=10_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9', 'batch=5_epochs=10_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20', 'batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20']
batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_

In [0]:
session_manager = training.SessionManager(base_dir)
model_dir = os.path.join(base_dir, "models")

qc_dir = os.path.join(base_dir, "qc")
qc_chunk_dir = os.path.join(qc_dir, "manual_labels")
window_path = os.path.join(qc_dir, "chunk_windows.json")

metadata_dir = os.path.join(base_dir, "Metadata")
training_set_path=os.path.join(metadata_dir, 'train.txt')
val_set_path=os.path.join(metadata_dir, 'val.txt')
test_set_path=os.path.join(metadata_dir, 'test.txt')

model_path = session_paths['session_dir']

model_name = "epoch26.h5"

results_dir = os.path.join(base_dir, "results", session_name, os.path.splitext(model_name)[0])

test_set_path = "/content/drive/My Drive/Metadata/manual_chunks.csv"

input_settings = session_manager.parse_component(selected_input_component)
bands = [int(i) for i in input_settings['bands']]

image_shape = (512, 512, len(bands))

callback_settings = session_manager.parse_component(selected_callback_component)
# batch_size must be 5 so that all image chunks are used in testing
batch_size = 5
num_classes = 2

test_set_list = training.get_image_list(test_set_path)

test_set_size = len(test_set_list)

In [52]:
print(results_dir)
print(bands)

/content/drive/My Drive/results/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20/epoch26
[1, 4, 8]


In [0]:
def data_gen(metadata_file_path, bands, batch_size):
    image_list = np.asarray(training.get_image_list(metadata_file_path))
    # np.random.seed(1)
    # np.random.shuffle(image_list)

    band_normalization_map = []
    counter = 0

    total_steps = image_list.shape[0] // batch_size
    while True:
        step_start = counter * batch_size
        step_end = step_start + batch_size
        images = []
        masks = []
        for j in range(step_start, step_end):
            images.append(np.load(image_list[j, 0])[:,:,bands])
            masks.append(np.load(image_list[j, 1]))

        y = to_categorical(np.array(masks), 2)
        yield np.array(images), y.reshape((batch_size, y.shape[1] * y.shape[2], y.shape[3]))

        counter +=1

        if counter >= total_steps:
            counter = 0

#Predictor


In [54]:
tic = time.time()
print("start time: {}".format(tic))
current_model = os.path.join(model_path, model_name)
if not os.path.exists(current_model):
    print("current_model does not exist\n", current_model)
    sys.exit(1)
toc = time.time()

test_data = data_gen(test_set_path, bands, batch_size)
toc = time.time() - toc
print(f"test_data loaded in {toc:.2f} seconds\ntotal runtime: {time.time() - tic:.2f}")

print(current_model)
my_model = load_model(current_model, 
                      custom_objects={"MaxPoolingWithArgmax2D": MaxPoolingWithArgmax2D, 
                                      "MaxUnpooling2D": MaxUnpooling2D,
                                      },
                      compile=False)
toc = time.time() - toc
print(f"model created in {toc:.2f} seconds\ntotal runtime: {time.time() - tic:.2f}")

probs = my_model.predict(test_data, steps = test_set_size // batch_size, verbose=1)

toc = time.time() - toc
print(f"predictions generated in {toc:.2f} seconds\ntotal runtime: {time.time() - tic:.2f}")

probs = probs.reshape((probs.shape[0], image_shape[0], image_shape[1], num_classes))
output = probs.argmax(axis=-1)
hist = np.histogram(output, bins=[0,1,2])
print(hist)

start time: 1590799769.1138177
test_data loaded in 0.00 seconds
total runtime: 0.00
/content/drive/My Drive/models/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20/epoch26.h5
model created in 1590799772.55 seconds
total runtime: 3.44
6/6 [==============================] - 112s 19s/step
predictions generated in 112.78 seconds
total runtime: 116.22
(array([6505929, 1358391]), array([0, 1, 2]))


In [55]:
test_scene_dir = "/content/drive/My Drive/test_chunks"
output_dir = "/content/drive/My Drive/output"

session_output_dir = os.path.join(output_dir, session_name, os.path.splitext(model_name)[0])
if not os.path.exists(session_output_dir):
  test = os.makedirs(session_output_dir)

test_scene_ids = [i for i in os.listdir(test_scene_dir) if os.path.isdir(os.path.join(test_scene_dir, i))]
print(test_scene_ids)
for i in test_scene_ids:
  output_scene_dir = os.path.join(session_output_dir, i)
  if not os.path.exists(output_scene_dir):
    os.mkdir(output_scene_dir)
    print("output_dir made at {}".format(output_scene_dir))

['LC82201072015017LGN00', 'LC82071182013336LGN00', 'LC81431082013352LGN00', 'LC81681112014002LGN00', 'LC80631112014002LGN00', 'LC80311222014338LGN00', 'LC81041072013303LGN00', 'LC82091172014001LGN00', 'LC82131132013362LGN00']
output_dir made at /content/drive/My Drive/output/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20/epoch26/LC82201072015017LGN00
output_dir made at /content/drive/My Drive/output/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20/epoch26/LC82071182013336LGN00
output_dir made at /content/drive/My Drive/output/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=5

In [56]:
print(output.shape)

(30, 512, 512)


In [57]:
for i in range(output.shape[0]):
    chunk_path, chunk_name = os.path.split(test_set_list[i][0])
    scene_chunk_path, scene_id = os.path.split(chunk_path)
    name, ext = os.path.splitext(chunk_name)
    out_file_name = f"{name}_output{ext}"
    print(os.path.join(session_output_dir, scene_id, out_file_name))
    np.save(os.path.join(session_output_dir, scene_id, out_file_name), output[i])

/content/drive/My Drive/output/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20/epoch26/LC80311222014338LGN00/chunk_6_1_output.npy
/content/drive/My Drive/output/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20/epoch26/LC80311222014338LGN00/chunk_6_2_output.npy
/content/drive/My Drive/output/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20/epoch26/LC80311222014338LGN00/chunk_7_1_output.npy
/content/drive/My Drive/output/dataset=one_percent_bands=1-4-8-_weights=n